Doesn't work, waiting for alignment model

In [1]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4

In [2]:
def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          



def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        print(sentences)
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

In [46]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('../lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

In [4]:
f = open('../../Latin/Livy/opensource/livy.foster01-02_lat.xml')
ltext = f.read()
f.close()
f = open('../../Latin/Livy/opensource/livy.foster01-02.xml')
etext = f.read()
f.close()

lat_sections = re.split(r'<div1 type="book" n="\d\d?\d?">', ltext)
eng_sections = re.split(r'<div1 type="book" n="\d\d?\d?">', etext)

In [5]:
lchaps = []
for book in lat_sections[1:]:
    sects = re.split(r'<milestone unit="chapter" n="\d\d?\d?"/>', book)[1:]
    lchaps.append(sects)

In [6]:
echaps = []
for book in eng_sections[1:]:
    sects = re.split(r'<milestone unit="chapter" n="\d\d?\d?"/>', book)[1:]
    echaps.append(sects)

In [31]:
len(echaps[0])

60

In [32]:
len(lchaps[0])

60

In [21]:
echaps[0]

['\n\t\t\t\t\t<milestone unit="section" n="1"/>First of all, then, it is generally agreed that when Troy was taken vengeance was wreaked upon the other Trojans, but that two, Aeneas and Antenor, were spared all the penalties of war by the Achivi, owing to long-standing claims of hospitality, and because they had always advocated peace and the giving back of Helen.\n\t\t\t\t\t<milestone unit="section" n="2"/> They then experienced various vicissitudes. Antenor, with a company of Eneti who had been expelled from Paphlagonia in a revolution and were looking for a home and a leader &mdash;for they had lost their king, Pylaemenes, at Troy<note>See the <hi rend="italics">Iliad,</hi> v. 576.</note> &mdash;came to the inmost bay of the Adriatic.\n\t\t\t\t\t<milestone unit="section" n="3"/> There, driving out the Euganei, who dwelt between the sea and the Alps, the Eneti and Trojans took possession of those lands. And in fact the place where they first landed is called Troy, and the district is

In [22]:
lsects = []
for chap in lchaps:
    for sec in chap:
        sects = re.split(r'<milestone unit="section" n="\d\d?\d?"/>', sec)[1:]
        lsects.append(sects)

In [23]:
esects = []
for chap in echaps:
    for sec in chap:
        sects = re.split(r'<milestone unit="section" n="\d\d?\d?"/>', sec)[1:]
        esects.append(sects)

In [75]:
esects[0][0]

'First of all, then, it is generally agreed that when Troy was taken vengeance was wreaked upon the other Trojans, but that two, Aeneas and Antenor, were spared all the penalties of war by the Achivi, owing to long-standing claims of hospitality, and because they had always advocated peace and the giving back of Helen.\n\t\t\t\t\t'

In [79]:
lsects[0][0]

' <reg>iam</reg> primum omnium satis constat Troia capta\n\t\t\t\t\tin ceteros saevitum esse Troianos: duobus, Aeneae\n\t\t\t\t\tAntenorique, et vetusti iure hospitii et quia pacis\n\t\t\t\t\treddendaeque Helenae semper auctores fuerunt,\n\t\t\t\t\tomne ius belli Achivos abstinuisse;'

In [77]:
lat_sections[1]

'<head>Liber I</head>\n\t\t\t\t<p><milestone unit="chapter" n="pr"/><milestone unit="section" n="1"/><milestone unit="section" n="1"/> <reg>facturusne</reg> operae pretium sim,<note>operae pretium sirm <hi rend="italics"><reg>sabellicus</reg> (from Quint. </hi>ix. iv. 74): sim operae pretium <foreign lang="greek">W.</foreign></note> si a primordio\n\t\t\t\t\turbis res populi Romani perscripserim, nec satis scio,\n\t\t\t\t\tnec,<milestone unit="section" n="2"/> si sciam, dicere ausim, quippe qui cum veterem\n\t\t\t\t\ttum volgatani esse rein videam, dum novi semper\n\t\t\t\t\tscriptores aut in rebus certius aliquid allaturos se\n\t\t\t\t\taut scribendi arte rudem vetustatem superaturos\n\t\t\t\t\tcredunt.<milestone unit="section" n="3"/> <reg>utcumque</reg> erit, iuvabit tamen rerum\n\t\t\t\t\tgestarum memoriae principis terrarum populi pro\n\t\t\t\t\tvirili parte et ipsum consuluisse; et si in tanta\n\t\t\t\t\tscriptorum turba mea fama in obscuro sit, nobilitate\n\t\t\t\t\tac magnitudi

all the sections seem to match up, so we can just proceed as usual

In [26]:
eng_text = []

for s in eng_sections:
    soup = BeautifulSoup(s,'lxml')
    if soup.find('note'):
        s = re.sub(r'<note>.*</note>', " ", s,flags=re.DOTALL)
        soup = BeautifulSoup(s,'lxml')
        eng_text.append(soup.text)
    else:
        eng_text.append(soup.text)

lat_text = []

for s in lat_sections:
    soup = BeautifulSoup(s,'lxml')
    if soup.find('note'):
        s = re.sub(r'<note>.*</note>', " ", s,flags=re.DOTALL)
        soup = BeautifulSoup(s,'lxml')
        lat_text.append(soup.text)
    else:
        lat_text.append(soup.text)

eng_text = eng_text[1:]
lat_text = lat_text[1:]

alignments = []

for c in range(len(eng_text)):
    es = ' '.join(eng_text[c].replace('\n',' ').split())
    ls = lat_text[c].replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    es = re.sub(r'\. \. \.','',es)
    ls = re.sub('[:;,]','',ls)
    es = re.sub('[:;,]','',es)

    epar = sent_tokenize(es)
    lpar = s_tokenizer.tokenize_sentences(ls)
    alignments.append(get_sec_alignments(lpar,epar))

In [28]:
alignments[0]

[('Liber I \t\t\t\t facturusne operae pretium sim  redegit.',
  ['Livy from the Founding of the City Book I Whether I am likely to accomplish anything worthy of the labour if I record the achievements of the Roman people from the foundation of the city I do not really know nor if I knew would I dare to avouch it perceiving as I do that the theme he assigned them the Aventine Hill planted a colony at Ostia extended the boundaries and revived the ceremonies established by Numa.']),
 ('huius filiis Delphos profectis et consulentibus quis eorum Romae regnaturus \t\t\t\t\t \t\t\t\t\tesset dictum est eum regnaturum qui primum matrem \t\t\t\t\tosculatus esset.',
  ['It was he who is said to have asked the augur Attus Navius to test his skill whether the thing he was thinking of could be accomplished and when Attus replied that it could to have bid him cut a whetstone in two with a razor Attus is said forthwith to have done.']),
 ('quod responsum cum ipsi aliter interpretarentur Iunius Brutus 

In [35]:
eng_text = []

for t in esects:
    text = []
    for s in t:
        soup = BeautifulSoup(s,'lxml')
        if soup.find('note'):
            s = re.sub(r'<note>.*</note>', " ", s,flags=re.DOTALL)
            soup = BeautifulSoup(s,'lxml')
            text.append(soup.text)
        else:
            text.append(soup.text)
    eng_text.append(text)

lat_text = []

for t in lsects:
    text = []
    for s in t:
        soup = BeautifulSoup(s,'lxml')
        if soup.find('note'):
            s = re.sub(r'<note>.*</note>', " ", s,flags=re.DOTALL)
            soup = BeautifulSoup(s,'lxml')
            text.append(soup.text)
        else:
            text.append(soup.text)
    lat_text.append(text)


In [36]:
lat_text[0][0]

'iam primum omnium satis constat Troia capta\n\t\t\t\t\tin ceteros saevitum esse Troianos: duobus, Aeneae\n\t\t\t\t\tAntenorique, et vetusti iure hospitii et quia pacis\n\t\t\t\t\treddendaeque Helenae semper auctores fuerunt,\n\t\t\t\t\tomne ius belli Achivos abstinuisse;'

In [39]:
eng_text[0]

['First of all, then, it is generally agreed that when Troy was taken vengeance was wreaked upon the other Trojans, but that two, Aeneas and Antenor, were spared all the penalties of war by the Achivi, owing to long-standing claims of hospitality, and because they had always advocated peace and the giving back of Helen.\n\t\t\t\t\t',
 'They then experienced various vicissitudes. Antenor, with a company of Eneti who had been expelled from Paphlagonia in a revolution and were looking for a home and a leader —for they had lost their king, Pylaemenes, at Troy  —came to the inmost bay of the Adriatic.\n\t\t\t\t\t',
 'There, driving out the Euganei, who dwelt between the sea and the Alps, the Eneti and Trojans took possession of those lands. And in fact the place where they first landed is called Troy, and the district is therefore known as Trojan, while the people as a whole are called the Veneti.\n\t\t\t\t\t',
 'Aeneas, driven from home by a similar misfortune, but guided by fate to undert

In [165]:
lpar

['bello deinde Aborigines Troianique simul \t\t\t\t\tpetiti.',
 'Turnus rex Rutulorum cui pacta Lavinia \t\t\t\t\tante adventum Aeneae fuerat praelatum sibi \t\t\t\t\t \t\t\t\t\tadvenam aegre patiens simul Aeneae Latinoque bellum \t\t\t\t\tintulerat.']

In [66]:
eng_sections[1]

'<head>Livy from the Founding of the City: Book I</head>\n\t\t\t\t<p><milestone unit="chapter" n="pr"/>\n\t\t\t\t\t<milestone unit="section" n="1"/>Whether I am likely to accomplish anything worthy of the labour, if I record the achievements of the Roman people from the foundation of the city, I do not really know, nor if I knew would I dare to avouch it;\n\t\t\t\t\t<milestone unit="section" n="2"/> perceiving as I do that the theme<note>Some scholars take <hi rend="italics">rem</hi> to mean <quote>the practice,</quote> <hi rend="italics">sc.</hi> of expressing confidence in one\'s ability.</note> is not only old but hackneyed, through the constant succession of new historians, who believe either that in their facts they can produce more authentic information, or that in their style they will prove better than the rude attempts of the ancients.\n\t\t\t\t\t<milestone unit="section" n="3"/> Yet, however this shall be, it will be a satisfaction to have done myself as much as lies in me to

In [63]:
eng_text[0][2]

'There, driving out the Euganei, who dwelt between the sea and the Alps, the Eneti and Trojans took possession of those lands. And in fact the place where they first landed is called Troy, and the district is therefore known as Trojan, while the people as a whole are called the Veneti.\n\t\t\t\t\t'

In [62]:
lat_text[0][1]

'casibus deinde\n\t\t\t\t\tvariis Antenorem cum multitudine Enetum, qui\n\t\t\t\t\tseditione ex Paphlagonia pulsi et sedes et ducem\n\t\t\t\t\trege Pylaemene ad Troiam amisso quaerebant,\n\t\t\t\t\tvenisse in intimum maris Hadriatici sinum, euganeisque, qui inter mare Alpesque incolebant, pulsis,\n\t\t\t\t\tEnetos Troianosque eas tenuisse terras. et in quem\n\t\t\t\t\tprimum egressi sunt locum Troia vocatur, pagoque\n\t\t\t\t\tinde Troiano nomen est: gens universa Veneti\n\t\t\t\t\tappellati.'

In [80]:
alignments = []


for s in range(len(eng_text)):
    for c in range(len(eng_text[s])):
        try:
            es = ' '.join(eng_text[s][c].replace('\n',' ').split())
            ls = lat_text[s][c].replace('\n', ' ')
        except:
            print(s,c,len(eng_text[s]),len(lat_text[s]))
            continue
        ls = re.sub(r'\. \. \.','',ls)
        es = re.sub(r'\. \. \.','',es)
        ls = re.sub('[:;,]','',ls)
        es = re.sub('[:;,]','',es)

        
        epar = sent_tokenize(es)
        lpar = s_tokenizer.tokenize_sentences(ls)
        alignments.append(get_sec_alignments(lpar,epar))

0 10 11 10
6 14 15 14
7 6 7 6
11 9 10 9
12 7 8 7
15 7 8 7
17 9 10 9
20 5 6 5
28 5 6 5
32 8 9 8
33 11 12 11
37 6 7 6
39 6 7 6
44 6 7 6
48 7 9 7
48 8 9 7
54 8 9 8
55 12 13 12
56 9 11 9
56 10 11 9
57 11 12 11
60 10 11 10
61 9 11 9
61 10 11 9
63 6 7 6
65 10 11 10
68 6 8 6
68 7 8 6
69 12 13 12
70 9 10 9
71 12 16 12
71 13 16 12
71 14 16 12
71 15 16 12
78 9 10 9
79 10 13 10
79 11 13 10
79 12 13 10
90 10 11 10
92 10 11 10
93 11 12 11
96 8 9 8
98 11 12 11
99 13 14 13
109 9 11 9
109 10 11 9
110 8 9 8
113 9 10 9
115 15 16 15
118 9 11 9
118 10 11 9
120 8 9 8


In [53]:
es

"After a blockade of a few days the place surrendered the besiegers had not delivered any new attack but the Volsci had lost heart from the moment of their unsuccessful battle and the capture of their camp. Summary of book II BRUTUS bound the people with an oath to allow no one to reign in Rome. Tarquinius Collatinus his colleague who had incurred suspicion because of his relationship to the Tarquinii he forced to abdicate the consulship and withdraw from the state. He ordered the king's goods to be plundered and consecrated his land to Mars. It was named the Campus Martius. Certain noble youths—among them his own sons and his brother's—he beheaded because they had conspired to bring back the kings. To the slave who gave the information a man called Vindicius he gave his freedom from his name came the word vindicta. Having led an army against the princes who had collected forces from Veii and Tarquinii and begun a war he fell in the battle together with Arruns the son of Superbus and t

In [88]:
alignments[-6]

[('quamquam cessere \t\t\t\t\tmagis quam pulsi hostes sunt quia ab tergo erant \t\t\t\t\tclivi in quos post principia integris ordinibus tutus \t\t\t\t\treceptus fuit.',
  'The Volsci weary from standing and from loss of sleep were driven back at the first assault though it was rather a retreat than a rout for behind them were hills to which under cover of the first line they withdrew safely and in good order.'),
 ('Consul ubi ad iniquum locum ventum \t\t\t\t\test sistit aciem.',
  'The consul ordered a halt when his army reached rising ground.'),
 ('miles aegre teneri clamare et \t\t\t\t\tposcere ut perculsis instare liceat.',
  'The infantry could hardly be restrained noisily demanding permission to press on after the fleeing enemy.')]

In [88]:
lat_text[1]

['',
 'bello deinde Aborigines Troianique simul\n\t\t\t\t\tpetiti. Turnus, rex Rutulorum, cui pacta Lavinia\n\t\t\t\t\tante adventum Aeneae fuerat, praelatum sibi\n\t\t\t\t\t\n\t\t\t\t\tadvenam aegre patiens simul Aeneae Latinoque bellum\n\t\t\t\t\tintulerat.',
 'neutra acies laeta ex eo certamine abiit:\n\t\t\t\t\tvicti Rutuli: victores Aborigines Troianique ducem\n\t\t\t\t\tLatinum amisere.',
 'inde Turnus Rutulique diffisi\n\t\t\t\t\trebus ad florentes opes Etruscorum Mezentiumque\n\t\t\t\t\tregem eorum confugiunt, qui Caere opulento tum\n\t\t\t\t\toppido imperitans, iam inde ab initio minime laetus\n\t\t\t\t\tnovae origine urbis, et tum nimio plus quam satis\n\t\t\t\t\ttutum esset accolis rem Troianam crescere ratus,\n\t\t\t\t\thaud gravatim socia arma Rutulis iunxit.',
 'Aeneas,\n\t\t\t\t\tadversus tanti belli terrorem ut animos Aboriginum\n\t\t\t\t\tsibi conciliaret, nec sub eodem iure solum sed etiam\n\t\t\t\t\tnomine omnes essent, Latinos utramque gentem\n\t\t\t\t\tappellavit.'